### Étape 1 : importation des library et tele chargement des données


on commence par importer les library necessaire et télécharger les données historiques des cours des 30 actions composant l'indice Dow Jones Industrial Average (DJIA). Nous utilisons 'yfinance' pour obtenir les cours de clôture ajustés

In [28]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import cvxpy as cp
import seaborn as sns

# set visual style
plt.style.use('seaborn-v0_8')

In [29]:
tickers = ['SPY', 'TLT', 'GLD', 'QQQ', 'EEM']

# Download adjusted close prices
star_date = '2019-01-01'
end_date = '2024-12-31'

prices_data = yf.download(tickers, start=star_date, end=end_date, multi_level_index = False)['Close']
prices_data = prices_data.dropna()
prices_data.head()

/tmp/ipython-input-1960456380.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  5 of 5 completed


Ticker,EEM,GLD,QQQ,SPY,TLT
Date,,,,,
2019-01-02,33.795906,121.330002,148.418488,225.660110,101.676346
2019-01-03,33.183167,122.430000,143.569595,220.275269,102.833382
2019-01-04,34.253315,121.440002,149.712158,227.653580,101.643089
2019-01-07,34.330990,121.860001,151.494583,229.448517,101.343361
2019-01-08,34.460438,121.529999,152.864929,231.604279,101.077057


### Étape 2 : calculer les rendements quotidiens, les rendement moyens et la covariance

Nous calculons les rendements quotidiens à partir des données sur les prix. Ceux-ci sont utilisés pour calculer les rendements moyens annualisés et la matrice de covariance, qui servent de données d'entrée pour nos simulations de portefeuille.

In [30]:
# Calculer les rendements quotidiens
log_returns = np.log(prices_data / prices_data.shift(1)).dropna()

# Calculer les rendements moyens annualisés et la matrice de covariance
mean_returns = log_returns.mean() * 252  # 252 jours de trading par an
cov_matrix = log_returns.cov() * 252  # Covariance annualisée

In [31]:
mean_returns

,0
Ticker,
EEM,0.034547
GLD,0.114427
QQQ,0.207473
SPY,0.158644
TLT,-0.029950


In [32]:
cov_matrix

Ticker,EEM,GLD,QQQ,SPY,TLT
Ticker,,,,,
EEM,0.046219,0.007029,0.038252,0.032686,-0.004628
GLD,0.007029,0.022402,0.004917,0.003677,0.007560
QQQ,0.038252,0.004917,0.059581,0.045312,-0.004580
SPY,0.032686,0.003677,0.045312,0.039683,-0.005812
TLT,-0.004628,0.007560,-0.004580,-0.005812,0.029263


### Étape 3 : Simuler des portfeuilles ( vente à decouvert autorisés)

Nous simulons un grand nombre de portefeuille aléatoire en générant des pondérations à l'aide de `np.random.randn()`.
cela permet à la vente de découvert; Certaines pondérations peuvent être négatives.
le rendement, la volatilité et le sharp-ratio de chaque portefeuille seront calculés et stockés

In [33]:
# définir une graine pour la reproductibilité
np.random.seed(2)

# Nombre de portefeuilles à simuler
num_portfolios = 10000
n_assets = len(tickers)

# stocker les résultats
results = {
    'Returns': [],
    'Volatility': [],
    'Sharpe Ratio': [],
    'Weights': []
}

# Simuler les portefeuilles
for _ in range(num_portfolios):
    # Générer des pondérations aléatoires (vente à découvert autorisée)
    weights = np.random.randn(n_assets)
    weights /= np.sum(np.abs(weights))  # Normaliser pour que la somme des valeurs absolues soit égale à 1

    # Calculer le rendement et la volatilité du portefeuille
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Calculer le ratio de Sharpe (en supposant un taux sans risque de 0%)
    sharpe_ratio = portfolio_return / portfolio_volatility

    # Stocker les résultats
    results['Returns'].append(portfolio_return)
    results['Volatility'].append(portfolio_volatility)
    results['Sharpe Ratio'].append(sharpe_ratio)
    results['Weights'].append(weights)

In [34]:
portfolios_df = pd.DataFrame(results)

In [35]:
portfolios_df

,Returns,Volatility,Sharpe Ratio,Weights
0,-0.024840,0.070462,-0.352534,"[-0.06896622052556646, -0.009311187393166404, ..."
1,-0.081309,0.139767,-0.581745,"[-0.18472024162144407, 0.11035659946728961, -0..."
2,0.021237,0.090719,0.234098,"[0.12140721223726011, 0.5046487326657839, 0.00..."
3,0.040076,0.051016,0.785558,"[-0.23404624615771136, -0.007510438728686587, ..."
4,-0.054119,0.118577,-0.456401,"[-0.33532033307125586, -0.059737030632453586, ..."
...,...,...,...,...
9995,0.024191,0.074682,0.323917,"[-0.41172080452544274, 0.19499866516717118, -0..."
9996,0.119066,0.152338,0.781594,"[-0.009730217148250321, -0.012109820573411938,..."
9997,0.028226,0.072002,0.392019,"[-0.014727840100086084, 0.3768618066027439, -0..."
9998,0.003094,0.080805,0.038294,"[0.1939126460114278, 0.4586796381192296, -0.16..."


### Étape 4 : Visualiser la frontière d'efficience brute

Nous convertissons nos résultats simulés en un DataFrame et utilisons matplotlib pour visualiser l'espace rendement-risque.
L'intensité des couleurs reflète le ratio de Sharpe.

In [36]:
fig = px.scatter(
    portfolios_df,
    x = 'Volatility',
    y = 'Returns',
    color = 'Sharpe Ratio',
    color_continuous_scale = 'viridis',
    title = 'frontière Éfficiente (shorting allowed)',
    labels = {'return': 'Rendement Espéré', 'volatilité': 'Risk Volatility'},
    width = 850,
    height = 500
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.show()

In [37]:
# trouver le portefeuille à ratio de sharp maximal
max_sharpe_idx = portfolios_df['Sharpe Ratio'].idxmax()
max_point = portfolios_df.loc[max_sharpe_idx]

# ajouter le meme graphe
fig.add_scatter(
    x = [max_point['Volatility']],
    y = [max_point['Returns']],
    mode = 'markers',
    marker = dict(size = 18, color = 'red', symbol = 'star'),
    name = 'Max Sharpe portfolio'
)

fig.show()

In [38]:
# Extraire le poids optimal
optimal_weights = portfolios_df.iloc[max_sharpe_idx]['Weights']
etf_name = tickers # En supposant que `tickers` soit notre liste d'ETF comme ['SPY', 'QQQ', 'GLD', 'TLT', 'EEM']

# visulalisons le poids optimal
fig_weights = px.bar(
    x = etf_name,
    y = optimal_weights,
    labels = {'x': 'ETF', 'y': 'Poids'},
    title = "Allocation d'actifs d'un portefeulle à ratio de sharpe maximal",
    width = 850,
    height = 500
)

fig_weights.update_yaxes(tickformat = '.0%')
fig_weights.show()

### Étape 5 : Simulation de la frontière efficiente « long only » (sans vente à découvert)

Nous simulons maintenant des portefeuilles soumis à une contrainte « long only ». Cela signifie que la pondération de tous les actifs doit être positive et égale à 1 ; en d'autres termes, aucune vente à découvert n'est autorisée.

Cela reflète des scénarios plus réalistes pour les portefeuilles de détail, les ETF ou les fonds « long only ».

In [39]:
# Stockage des résultats à long terme
results_long_only = {
    'Return': [],
    'Volatility': [],
    'Sharpe Ratio': [],
    'Weights': []
}

# Simuler des portefeuilles long terme
np.random.seed(42)
for _ in range(num_portfolios):
  weights = np.random.random(n_assets)  # tous positifs
  weights /= np.sum(weights)            # normaliser pour obtenir une somme égale à 1

  port_return = np.dot(weights, mean_returns)
  port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
  sharpe_ratio = port_return / port_volatility

  results_long_only['Return'].append(port_return)
  results_long_only['Volatility'].append(port_volatility)
  results_long_only['Sharpe Ratio'].append(sharpe_ratio)
  results_long_only['Weights'].append(weights)


In [40]:
# convertir les rultats en datframe
portfolios_long = pd.DataFrame(results_long_only)
portfolios_long

,Return,Volatility,Sharpe Ratio,Weights
0,0.129412,0.144256,0.897099,"[0.13319702814025883, 0.33810081711389406, 0.2..."
1,0.111281,0.150009,0.741831,"[0.06528491964469331, 0.02430844330237927, 0.3..."
2,0.141015,0.137759,1.023634,"[0.00928441856775223, 0.4374675865753444, 0.37..."
3,0.120897,0.146113,0.827418,"[0.10567348701744264, 0.17529742718559654, 0.3..."
4,0.076195,0.141243,0.539459,"[0.3279089323822976, 0.07475862795590853, 0.15..."
...,...,...,...,...
9995,0.108223,0.141618,0.764191,"[0.1597470525122923, 0.16501974409071296, 0.29..."
9996,0.111219,0.148950,0.746686,"[0.08999419626825104, 0.044998570507992594, 0...."
9997,0.075449,0.118031,0.639232,"[0.1866057391525586, 0.24367833570818154, 0.19..."
9998,0.095166,0.138405,0.687586,"[0.24664170718585862, 0.1847704022338319, 0.25..."


### Étape 6 : Visualisation de la frontière efficiente long-only (sans vente à découvert)

Maintenant que nous avons simulé des milliers de portefeuilles avec des contraintes long-only (sans vente à découvert), nous visualisons la frontière efficiente qui en résulte.

Cela montre des portefeuilles composés entièrement de pondérations positives, une configuration réaliste pour les fonds communs de placement, les ETF et les investisseurs individuels.

Nous mettons également en évidence le portefeuille présentant le ratio de Sharpe le plus élevé, qui représente le meilleur compromis entre risque et rendement dans l'univers des positions longues uniquement.


In [41]:
# trouver le portefeuille à ratio de sharp maximal
max_sharpe_idx_long = portfolios_long['Sharpe Ratio'].idxmax()
max_point_long = portfolios_long.iloc[max_sharpe_idx_long]

# Visualiser le frontier efficient (long-only)
fig_long = px.scatter(
    portfolios_long,
    x='Volatility',
    y='Return',
    color='Sharpe Ratio',
    color_continuous_scale='Viridis',
    title='Efficient Frontier (Long-Only)',
    labels={'Volatility': 'Risk (Volatility)', 'Return': 'Expected Return'},
    width=900,
    height=500
)

# Mettre en évidence le point Sharpe maximal
fig_long.add_scatter(
    x=[max_point_long['Volatility']],
    y=[max_point_long['Return']],
    mode='markers',
    marker=dict(size=12, color='red', symbol='star'),
    name='Max Sharpe Portfolio'
)

fig_long.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig_long.show()


### Étape 7 : Répartition des actifs du portefeuille Long-Only Max Sharpe

Nous visualisons maintenant les pondérations des actifs pour le portefeuille Long-Only optimal, celui qui présente le ratio de Sharpe le plus élevé.

Cela nous aide à comprendre comment le capital est réparti entre les ETF lorsque l'objectif est de maximiser le rendement par rapport au risque sans recourir à la vente à découvert.


In [42]:
# Extraire les pondérations optimales à partir d'une simulation à position longue uniquement
optimal_weights_long = portfolios_long.iloc[max_sharpe_idx_long]['Weights']

# Diagramme à barres représentant la répartition des actifs
fig_weights_long = px.bar(
    x=tickers,
    y=optimal_weights_long,
    labels={'x': 'ETF', 'y': 'Weight'},
    title='Asset Allocation of Long-Only Max Sharpe Portfolio',
    width=700,
    height=400
)
fig_weights_long.update_layout(yaxis_tickformat='.0%')
fig_weights_long.show()


### Étape 8 : Optimisation des portefeuilles à l'aide d'un solveur (CVXPY)

Nous passons maintenant de la simulation à une approche d'optimisation exacte à l'aide du solveur `cvxpy`.

Cette étape permet de déterminer le portefeuille présentant le **ratio de Sharpe maximal** à l'aide de l'optimisation matricielle. Les entrées (rendements attendus et covariance) doivent être des tableaux NumPy afin d'éviter tout problème de compatibilité avec le solveur.


In [43]:
# Convertir les données
mu = mean_returns.to_numpy()
cov = cov_matrix.to_numpy()
n_assets = len(mu)

# Variable d'optimisation
w = cp.Variable(n_assets)

# Définir le rendement et la volatilité du portefeuille
portfolio_return = mu @ w
portfolio_volatility = cp.quad_form(w, cov)

# Objectif reformulé : maximiser le rendement, sous réserve d'une volatilité = 1 (logique de Sharpe)
constraints = [
    cp.sum(w) == 1,                      # Investissement total
    portfolio_volatility <= 1           # Corriger ou limiter la volatilité
]

# Définir et résoudre le problème
prob = cp.Problem(cp.Maximize(portfolio_return), constraints)
prob.solve()

# Extraire les pondérations
opt_weights_sharpe = w.value

# Affichage
print("Solver status:", prob.status)
print("Optimal weights (max Sharpe, shorting allowed):")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {opt_weights_sharpe[i]:.4f}")


Solver status: optimal
Optimal weights (max Sharpe, shorting allowed):
SPY: -5.0712
TLT: 3.9383
GLD: 4.1342
QQQ: 0.8844
EEM: -2.8857


### Étape 9 : superposer le portefeuille optimal basé sur le solveur à la frontière efficiente

Maintenant que nous avons calculé le portefeuille optimal à l'aide d'un solveur (avec possibilité de vente à découvert), nous le représentons sous la forme d'un « X » rouge sur la frontière efficiente des 10 000 portefeuilles simulés.

Cette étape nous aide à comparer visuellement :

- Le **portefeuille optimal** (issu de l'optimisation mathématique) par rapport à l'ensemble des **portefeuilles simulés aléatoirement**
- Le **portefeuille maximisant le ratio de Sharpe** se trouve dans le coin supérieur gauche de la frontière, souvent au-delà de la dispersion si la vente à découvert est largement utilisée
- Comment les méthodes basées sur la simulation peuvent passer à côté du meilleur point lorsque le nombre de portefeuilles est limité (par exemple, 10 000)

Cette comparaison visuelle permet de passer naturellement aux **contraintes du monde réel**, que nous commencerons à appliquer ensuite.


In [44]:
# Create DataFrame of simulated portfolios (already exists as portfolios_df)
# portfolios_df['Return'], portfolios_df['Volatility'], portfolios_df['Sharpe']

# Calculate return and volatility of optimal portfolio from solver
opt_return = np.dot(opt_weights_sharpe, mean_returns)
opt_volatility = np.sqrt(np.dot(opt_weights_sharpe.T, np.dot(cov_matrix, opt_weights_sharpe)))

# Plot
fig = px.scatter(
    portfolios_df,
    x='Volatility',
    y='Returns',
    color='Sharpe Ratio',
    color_continuous_scale='Viridis',
    title=('Frontière efficiente avec portefeuille optimal basé sur un solveur (vente à découvert autorisée)'),
    labels={'Volatility': 'Risque (Volatilité)', 'Return': 'Rendement'},
    width=800,
    height=500
)

# Add red marker for optimal portfolio
fig.add_scatter(
    x=[opt_volatility],
    y=[opt_return],
    mode='markers',
    marker=dict(size=12, color='red', symbol='x'),
    name='Solver Optimal Portfolio'
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()


### Étape 10 : Visualiser les pondérations optimales du portefeuille (à l'aide d'un solveur, vente à découvert autorisée)

Maintenant que nous avons déterminé le **portefeuille optimal selon le ratio de Sharpe** à l'aide d'un optimiseur mathématique (avec vente à découvert autorisée), nous visualisons les allocations d'actifs qui en résultent.

Points clés à observer :

- **Certaines pondérations peuvent être négatives**, indiquant des positions courtes dans certains ETF.
- Quelques actifs peuvent présenter des allocations positives ou négatives importantes, ce qui peut être irréaliste dans la pratique sans contraintes de levier ou d'emprunt.
- Cela souligne pourquoi il est nécessaire d'introduire des contraintes du monde réel, ce que nous ferons à l'étape suivante.

Le graphique à barres ci-dessous montre la pondération exacte attribuée à chaque ETF dans le portefeuille optimisé.


In [45]:
# Créer un DataFrame pour les pondérations optimales (vente à découvert autorisée)
opt_weights_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': opt_weights_sharpe
}).sort_values(by='Weight', ascending=False)

# Tracer à l'aide de Plotly
fig = px.bar(round(opt_weights_df,2), x='Ticker', y='Weight',
             title="Pondérations optimales (Sharpe maximal, vente à découvert autorisée)",
             labels={'Weight': 'pondération du portefeuille'},
             text='Weight')

fig.update_layout(template='plotly_white', yaxis_tickformat=".0%")
fig.show()


### Étape 11 : Optimisation du portefeuille (contrainte « long only », basée sur un solveur)

Auparavant, nous autorisions la **vente à découvert**, ce qui pouvait entraîner des **pondérations irréalistes**, voire un effet de levier. Nous allons maintenant répéter l'optimisation, mais en **limitant les pondérations à des valeurs non négatives**, c'est-à-dire des positions « long only ».

Points clés :
- Cela simule un **scénario réaliste pour un investisseur particulier** où seules les positions positives sont autorisées.
- L'optimiseur va maintenant rechercher le portefeuille présentant le **ratio de Sharpe maximal** dans le cadre de cette contrainte.
- La solution se situera probablement dans le nuage de la **frontière efficiente** et semblera plus équilibrée.

Ensuite, nous résolvons cela à l'aide de `cvxpy`, puis nous visualisons l'allocation efficiente qui en résulte.


In [46]:
# Définir la variable CVXPY
w = cp.Variable(n_assets)

# Définir un rendement cible (peut être ajusté)
target_return = mean_returns.mean()

# Objectif : minimiser la variance du portefeuille
portfolio_variance = cp.quad_form(w, cov_matrix)
objective = cp.Minimize(portfolio_variance)

# Contraintes : position longue uniquement, investissement intégral, rendement minimum
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    mean_returns.values @ w >= target_return
]

# Résoudre le problème
problem = cp.Problem(objective, constraints)
problem.solve()

# Extraire les poids optimaux
opt_weights_long_only = w.value


In [47]:
# Afficher proprement
print("Poids optimaux uniquement longs:")
for ticker, weight in zip(tickers, opt_weights_long_only):
    print(f"{ticker}: {weight:.4f}")

Poids optimaux uniquement longs:
SPY: 0.0000
TLT: 0.4271
GLD: 0.0000
QQQ: 0.3463
EEM: 0.2266


### Étape 12 : Visualisation du portefeuille optimal long-only sur la frontière efficiente

Nous superposons maintenant le portefeuille optimal long-only sur le graphique de la frontière efficiente. Cela nous aide à comparer l'optimisation basée sur un solveur avec la frontière basée sur la simulation (qui incluait également des positions courtes).


In [48]:
# Calculer le rendement et la volatilité d'un portefeuille long-only optimal
opt_return_long_only = np.dot(opt_weights_long_only, mean_returns)
opt_volatility_long_only = np.sqrt(np.dot(opt_weights_long_only.T, np.dot(cov_matrix, opt_weights_long_only)))

# Graphique
fig = px.scatter(
    portfolios_long,
    x='Volatility',
    y='Return',
    color='Sharpe Ratio',
    color_continuous_scale='Viridis',
    title='Frontière efficiente avec portefeuille optimal long-only',
    labels={'Volatility': 'Risque (volatilité) ', 'Rendement ' : ' Rendement attendu'},
    width=800,
    height=500
)

# Ajouter un marqueur vert pour le portefeuille optimal à position longue uniquement
fig.add_scatter(
    x=[opt_volatility_long_only],
    y=[opt_return_long_only],
    mode='markers',
    marker=dict(size=25, color='red', symbol='star'),
    name='Portefeuille optimal long-only',
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
fig.show()


### Étape 13 : Visualisation des pondérations optimales du portefeuille (basée sur un solveur, uniquement à long terme)

Après avoir optimisé le portefeuille à l'aide de **CVXPY avec des contraintes uniquement à long terme**, nous visualisons maintenant les allocations d'actifs qui en résultent.

Observations clés :

- **Toutes les pondérations sont non négatives**, ce qui indique qu'il n'y a pas de vente à découvert  
- L'optimiseur trouve le portefeuille présentant le **ratio de Sharpe maximal** dans l'espace long-only réalisable  
- Cette version est plus **réaliste et plus facile à mettre en œuvre** pour la plupart des investisseurs particuliers et institutionnels  
- Par rapport au cas où la vente à découvert est autorisée, vous remarquerez une **allocation plus conservatrice et plus équilibrée**

Le graphique à barres ci-dessous illustre la répartition des pondérations entre les ETF dans le portefeuille optimal long-only.


In [49]:
# Créer un DataFrame pour les pondérations optimales (long-only)
opt_longonly_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': opt_weights_long_only
}).sort_values(by='Weight', ascending=False)

# Tracer à l'aide de Plotly
fig = px.bar(round(opt_longonly_df,2), x='Ticker', y='Weight',
             title="Pondérations optimales (Max Sharpe, Long-Only)",
             labels={'Weight': 'pondération du portefeuille'},
             text='Weight')

fig.update_layout(template='plotly_white', yaxis_tickformat=".0%")
fig.show()


### Étape 14 : Optimiser le portefeuille avec des limites de pondération et un seuil de rendement

Nous introduisons maintenant des **contraintes réalistes pour le portefeuille** :

- Toutes les pondérations doivent être comprises entre **5 % et 40 %** (afin d'éviter une exposition excessive ou des allocations proches de zéro)
- Le portefeuille doit être **entièrement investi** (la somme des pondérations doit être égale à 1)
- Le portefeuille doit atteindre un **rendement minimum attendu**

Cela permet de modéliser un scénario plus pratique et d'éviter des allocations trop concentrées ou négligeables.


In [50]:
# Optimisation basée sur un solveur avec limites d'allocation (long-only + plancher/plafond)

# Convertir les rendements moyens en tableau numpy
mu = mean_returns.values  # S'assurer que le tableau numpy est de type 1D
cov = cov_matrix.values   # Convertir également la matrice de covariance en numpy

# Définir la variable d'optimisation
w = cp.Variable(n_assets)

# Rendement et variance du portefeuille
portfolio_return = mu @ w
portfolio_variance = cp.quad_form(w, cov)



# Conntraints
target_return = 0.001  # # Ajustez ce seuil en fonction de vos attentes en matière de rendement
constraints = [
    cp.sum(w) == 1,        # Entièrement investi
    w >= 0.05,             # Allouation d'au moins 5 % à chaque actif
    w <= 0.4,              # Allocation maximale de 40 % pour chaque actif
    portfolio_return >= target_return  # Rendement minimum attendu

]

# Objectif : minimiser la variance sous contraintes
objective = cp.Minimize(portfolio_variance)

# Résoudre le problème
problem = cp.Problem(objective, constraints)
problem.solve()

# Extraire les poids optimaux
opt_weights_min_var_bounded = w.value


In [51]:
# Afficher proprement
print("Optimisation sous Contraintes de Rendement Pondérées :")
for ticker, weight in zip(tickers, opt_weights_min_var_bounded):
    print(f"{ticker}: {weight:.4f}")

Optimisation sous Contraintes de Rendement Pondérées :
SPY: 0.0500
TLT: 0.3246
GLD: 0.0500
QQQ: 0.2120
EEM: 0.3634


### Étape 15 : Visualisation du portefeuille contraint long-only sur la frontière efficiente

Au cours de cette étape, nous visualisons le portefeuille optimisé à l'aide d'un **objectif de variance minimale** avec une **contrainte de rendement minimum** et une **allocation long-only**.  
Ce point est représenté graphiquement sur les 10 000 portefeuilles aléatoires simulés précédemment.
\
**Observations clés** :
- Ce point reflète un portefeuille réaliste, sans vente à découvert et avec un rendement minimum requis.
- Vous pouvez maintenant comparer cette solution par rapport à la frontière efficiente aléatoire et à d'autres solutions optimisées.


In [52]:
# Calcul du rendement et de la volatilité du portefeuille optimal avec contrainte long-only
opt_ret_constrained = np.dot(opt_weights_min_var_bounded, mean_returns)
opt_vol_constrained = np.sqrt(np.dot(opt_weights_min_var_bounded.T, np.dot(cov_matrix, opt_weights_min_var_bounded)))

# Graphique
fig = px.scatter(
    portfolios_long,
    x='Volatility',
    y='Return',
    color='Sharpe Ratio',
    color_continuous_scale='Viridis',
    title="Frontière efficiente avec portefeuille optimal contraint à position longue uniquement",
    labels={'Volatility': 'Risque (volatilité) ', 'Rendement ' : ' Rendement attendu'},
    width=800,
    height=500
)

# Ajouter un marqueur étoile rouge pour le portefeuille contraint
fig.add_scatter(
    x=[opt_vol_constrained],
    y=[opt_ret_constrained],
    mode='markers',
    marker=dict(size=30, color='red', symbol='star'),
    name='Portefeuille optimal long-only',
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

### Étape 16 : Diagramme à barres des pondérations optimales du portefeuille long-only contraint

Maintenant que nous avons déterminé le **portefeuille à variance minimale** avec un **rendement minimum requis** et une **contrainte long-only**,  
nous représentons graphiquement les pondérations obtenues.

📌 **Observations clés** :
- Toutes les pondérations sont **positives**, conformément à la contrainte long-only.
- L'optimiseur a attribué des pondérations plus élevées aux actifs qui contribuent le moins à la volatilité du portefeuille tout en atteignant le rendement cible.


In [53]:
# Créer un DataFrame pour les poids optimaux sous contraints
constrained_weights_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': opt_weights_min_var_bounded
}).sort_values(by='Weight', ascending=False)

# Tracer à l'aide de Plotly
fig = px.bar(
    round(constrained_weights_df,2), x='Ticker', y='Weight',
    title="Pondérations optimales (variance minimale avec rendement plancher, long-only)",
    labels={'Weight': 'Poids du Portefeuille'},
    text='Weight'
)

fig.update_layout(template='plotly_white', yaxis_tickformat=".0%")
fig.show()


In [54]:
# Définir une fonction d'aide
def portfolio_stats(weights, mean_returns, cov_matrix, periods_per_year=252):
    port_return = np.dot(weights, mean_returns)  # pas de * periods_per_year
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))  # Pas de sqrt(252)
    sharpe = port_return / port_vol

    return port_return, port_vol, sharpe

# Stocker les résultats pour chaque portefeuille optimisé
results = []

# Portefeuille 1 : Solveur - Sharpe Maximal (Vente à découvert autorisée)
ret, vol, sharpe = portfolio_stats(opt_weights_sharpe, mean_returns, cov_matrix)
results.append(["Sharpe Maximal (Vente à découvert autorisée)", ret, vol, sharpe, *opt_weights_sharpe])

# Portefeuille 2 : Solveur - Sharpe Maximal (Long-only)
ret, vol, sharpe = portfolio_stats(opt_weights_long_only, mean_returns, cov_matrix)
results.append(["Sharpe Maximal (Long-only)", ret, vol, sharpe, *opt_weights_long_only])

# Portefeuille 3 : Solveur - Variance Minimale (Bornée, Plancher de rendement)
ret, vol, sharpe = portfolio_stats(opt_weights_min_var_bounded, mean_returns, cov_matrix)
results.append(["Variance Minimale (Rendement ≥ 0.1%, bornes 5-40%)", ret, vol, sharpe, *opt_weights_min_var_bounded])

# Créer le DataFrame
summary_df = pd.DataFrame(results, columns=[
    "Portefeuille", "Rendement Ann.", "Volatilité Ann.", "Ratio de Sharpe",
    "SPY", "QQQ", "GLD", "TLT", "EEM"
])

# Arrondir pour l'affichage
summary_df = summary_df.round(4)

# Afficher le résumé
summary_df

,Portefeuille,Rendement Ann.,Volatilité Ann.,Ratio de Sharpe,SPY,QQQ,GLD,TLT,EEM
0,Sharpe Maximal (Vente à découvert autorisée),1.3599,1.0000,1.3599,-5.0712,3.9383,4.1342,0.8844,-2.8857
1,Sharpe Maximal (Long-only),0.0970,0.1095,0.8862,0.0000,0.4271,0.0000,0.3463,0.2266
2,"Variance Minimale (Rendement ≥ 0.1%, bornes 5-...",0.0720,0.1075,0.6695,0.0500,0.3246,0.0500,0.2120,0.3634
